In [1]:
import time
import pandas as pd
import numpy as np

import requests

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
import selenium.webdriver.support.expected_conditions as EC
import selenium.webdriver.support.ui as ui

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException

from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup

chrome_options = Options()
chrome_options.add_argument('disable-infobars')

In [8]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

In [3]:
url = 'https://www.justdial.com/Hyderabad/Ready-Mix-Concrete-Dealers-in-Bachupally/nct-10993743'

In [4]:
def waitUntil(w,ec,by,key):
    
    try:
        return w.until(ec((by,key)))
    except StaleElementReferenceException:
        return False
    
    return False

ph_code = {'dc':'+','fe':'(','hg':')','ba':'-',
           'yz':'1','wx':'2','vu':'3','ts':'4','rq':'5','po':'6','nm':'7','lk':'8','ji':'9','acb':'0'}

In [24]:
def getData(divs):
    
    table = []

    #for i in divs[-10:]:
    for div in divs[-10:]:

        row = []

        content = str(div)
        #content = i.get_attribute('innerHTML')
        #div = BeautifulSoup(content, 'html.parser')
        
        name = div.find('span','lng_cont_name').get_text()
        votes = div.find('span','lng_vote').get_text().replace('Votes','').strip()
        
        rating = None
        if votes != '0':
            rating = div.find('span','green-box').get_text()
        
        address = None
        address_div = div.find('span','cont_fl_addr')
        if address_div != None:
            address = address_div.get_text()

        numbers_all = div.find('p','contact-info')
        number = None
        if numbers_all != None:
            numbers_all = numbers_all.findAll('span','mobilesv')
            number = ''
            for j in numbers_all:
                number += ph_code[j['class'][1].replace('icon-','')]

        verified = False
        
        if 'jd_verified' in content:
            verified = True

        trusted = False
        if 'jd_trusted' in content:
            trusted = True

        page_link = div.find('h2','store-name').find('a')['href']

        row.extend([name,rating,votes,number,address,verified,trusted,page_link])
        table.append(row)
        
    return table

In [6]:
driver = webdriver.Chrome('chromedriver.exe',options=chrome_options)
driver.set_window_size(1366,768)

driver.get(url)

startTime = time.time()

wait = ui.WebDriverWait(driver, 15)
waitUntil(wait,EC.presence_of_element_located,By.ID,'best_deal_div')
waitUntil(wait,EC.visibility_of_element_located,By.ID,'best_deal_div')

time.sleep(1)

driver.find_element_by_id('best_deal_div').find_element_by_class_name('jcl').click()

divs = driver.find_element_by_id('tab-5').find_elements_by_class_name('cntanr')

table = []

table.extend(getData(divs))

for i in range(1,12,1):

    driver.execute_script("arguments[0].scrollIntoView();", divs[-1])
    
    counter = 0
    
    count = len(divs)

    while True:
        try:
            if len(driver.find_element_by_id('tab-5').find_elements_by_class_name('cntanr')) > count:
                break
        except StaleElementReferenceException:
            None

        counter += 1

        if counter > 40:
            print('Network issue')
            networkIssue = True
            break

        time.sleep(0.250)

    time.sleep(1)
    
    divs = driver.find_element_by_id('tab-5').find_elements_by_class_name('cntanr')
    
    table.extend(getData(divs))
    
print(time.time() - startTime)

Network issue
Network issue
52.42992091178894


In [7]:
driver.quit()

In [8]:
dfObj = pd.DataFrame(table, columns = ['Name','Rating','Votes','Phone','Address','Verified','Trusted','Url']) 
dfObj

,Name,Rating,Votes,Phone,Address,Verified,Trusted,Url
0,Omega RMC,4.5,15,+(91)-9152182118,"Flat No.6-150, Near Gandimaisamma Chowrasta, B...",True,True,https://www.justdial.com/hyderabad/Omega-RMC-B...
1,Premier Readymix Concrete,4.7,22,+(91)-9152874536,"Plot No 57, Telcom Nagar, Gachibowli, Hyderaba...",False,False,https://www.justdial.com/Hyderabad/Premier-Rea...
2,Rdc Concrete India Pvt Ltd,4.1,16,+(91)-9152886244,"Plot No 8/A, Survey No 334 to 337, Bollaram Ro...",False,False,https://www.justdial.com/Hyderabad/Rdc-Concret...
3,Metro Rmc,4.0,21,+(91)-9152194066,"39-p , 1ST Floor , B R R Residency ,D Pochamp...",True,True,https://www.justdial.com/Hyderabad/Metro-Rmc-N...
4,G T Associates,4.5,22,+(91)-9152861461,"4-55, Kukatpally, Hyderabad - 500072, Near BJP...",False,False,https://www.justdial.com/Hyderabad/G-T-Associa...
5,Bmr Ready Mix Concrete,4.9,8,+(91)-9152596296,"201,Chaitanya Chambers, Chaitanyapuri, Hyderab...",False,False,https://www.justdial.com/hyderabad/Bmr-Ready-M...
6,Mega Ready Mix Concrete,4.5,3,+(91)-9152217638,"Survey No 5-53, Kolluru Village, Tellapur, Hyd...",False,False,https://www.justdial.com/Hyderabad/Mega-Ready-...
7,Global Rmc,4.5,30,+(91)-9152235557,"Sy No.294, Rampalli Village, Cherlapalli, Rang...",False,False,https://www.justdial.com/Hyderabad/Global-Rmc-...
8,Jaidhar Ready Mix Concrete,4.2,63,+(91)-9152188052,"Plot No 37, A Colony, Karkhana, Hyderabad - 50...",False,False,https://www.justdial.com/Hyderabad/Jaidhar-Rea...
9,Sln Ready Mix Concrete,4.3,6,+(91)-9152883544,"House No 8/73/20, ECIL, Hyderabad - 500062, Ne...",False,False,https://www.justdial.com/Hyderabad/Sln-Ready-M...


In [9]:
def getProducts(page_url):
    
    response = requests.get(page_url,headers=headers)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    content = soup.find('span','showmore').text
    
    content = ', '.join(content.split(' \r\n\t\t\t\t')).replace('\n','').replace('\r','').replace('\t','').strip()
    
    return content

In [10]:
startTime = time.time()
dfObj['Products'] = dfObj['Url'].apply(lambda x : getProducts(x))
print(time.time() - startTime)

106.99052834510803


In [11]:
dfObj

,Name,Rating,Votes,Phone,Address,Verified,Trusted,Url,Products
0,Omega RMC,4.5,15,+(91)-9152182118,"Flat No.6-150, Near Gandimaisamma Chowrasta, B...",True,True,https://www.justdial.com/hyderabad/Omega-RMC-B...,Ready Mix Concrete Dealers
1,Premier Readymix Concrete,4.7,22,+(91)-9152874536,"Plot No 57, Telcom Nagar, Gachibowli, Hyderaba...",False,False,https://www.justdial.com/Hyderabad/Premier-Rea...,"Ready Mix Concrete Dealers, Ready Mix Concrete..."
2,Rdc Concrete India Pvt Ltd,4.1,16,+(91)-9152886244,"Plot No 8/A, Survey No 334 to 337, Bollaram Ro...",False,False,https://www.justdial.com/Hyderabad/Rdc-Concret...,"Ready Mix Concrete Dealers, Ready Mix Concrete..."
3,Metro Rmc,4.0,21,+(91)-9152194066,"39-p , 1ST Floor , B R R Residency ,D Pochamp...",True,True,https://www.justdial.com/Hyderabad/Metro-Rmc-N...,"Ready Mix Concrete Dealers, Ready Mix Concrete..."
4,G T Associates,4.5,22,+(91)-9152861461,"4-55, Kukatpally, Hyderabad - 500072, Near BJP...",False,False,https://www.justdial.com/Hyderabad/G-T-Associa...,"Ready Mix Concrete Dealers, Cement Dealers-Ult..."
5,Bmr Ready Mix Concrete,4.9,8,+(91)-9152596296,"201,Chaitanya Chambers, Chaitanyapuri, Hyderab...",False,False,https://www.justdial.com/hyderabad/Bmr-Ready-M...,"Ready Mix Concrete Dealers, Ready Mix Concrete..."
6,Mega Ready Mix Concrete,4.5,3,+(91)-9152217638,"Survey No 5-53, Kolluru Village, Tellapur, Hyd...",False,False,https://www.justdial.com/Hyderabad/Mega-Ready-...,"Ready Mix Concrete Dealers, Ready Mix Concrete..."
7,Global Rmc,4.5,30,+(91)-9152235557,"Sy No.294, Rampalli Village, Cherlapalli, Rang...",False,False,https://www.justdial.com/Hyderabad/Global-Rmc-...,"Ready Mix Concrete Dealers, Ready Mix Concrete..."
8,Jaidhar Ready Mix Concrete,4.2,63,+(91)-9152188052,"Plot No 37, A Colony, Karkhana, Hyderabad - 50...",False,False,https://www.justdial.com/Hyderabad/Jaidhar-Rea...,"Concrete Mixer-Birla Readymix, Ready Mix Concr..."
9,Sln Ready Mix Concrete,4.3,6,+(91)-9152883544,"House No 8/73/20, ECIL, Hyderabad - 500062, Ne...",False,False,https://www.justdial.com/Hyderabad/Sln-Ready-M...,"Ready Mix Concrete Dealers, Ready Mix Concrete..."


In [2]:
import time

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

ph_code = {'dc':'+','fe':'(','hg':')','ba':'-',
           'yz':'1','wx':'2','vu':'3','ts':'4','rq':'5','po':'6','nm':'7','lk':'8','ji':'9','acb':'0'}

In [4]:
url = 'https://www.justdial.com/Hyderabad/Ready-Mix-Concrete-Dealers-in-Bachupally/nct-10993743'

In [5]:
def getData(divs):
    
    table = []

    for div in divs:

        row = []

        content = str(div)
        
        name = div.find('span','lng_cont_name').get_text()
        votes = div.find('span','lng_vote').get_text().replace('Votes','').strip()
        
        rating = None
        if votes != '0':
            rating = div.find('span','green-box').get_text()
        
        address = None
        address_div = div.find('span','cont_fl_addr')
        if address_div != None:
            address = address_div.get_text()

        numbers_all = div.find('p','contact-info')
        number = None
        if numbers_all != None:
            numbers_all = numbers_all.findAll('span','mobilesv')
            number = ''
            for j in numbers_all:
                number += ph_code[j['class'][1].replace('icon-','')]

        verified = False
        
        if 'jd_verified' in content:
            verified = True

        trusted = False
        if 'jd_trusted' in content:
            trusted = True

        page_link = div.find('h2','store-name').find('a')['href']

        row.extend([name,rating,votes,number,address,verified,trusted,page_link])
        table.append(row)
        
    return table

In [6]:
table = []
i = 0

while True:
    i += 1
    
    response = requests.get(url+'/page-'+str(i),headers=headers)
    page = BeautifulSoup(response.text, 'html.parser')
    divs = page.findAll('li','cntanr')
    
    if len(divs) == 0:
        break
    
    table.extend(getData(divs))
    
    if len(divs) < 10:
        break

ConnectionError: HTTPSConnectionPool(host='www.justdial.com', port=443): Max retries exceeded with url: /Hyderabad/Ready-Mix-Concrete-Dealers-in-Bachupally/nct-10993743/page-1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x110add4a8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [28]:
dfObj = pd.DataFrame(table, columns = ['Name','Rating','Votes','Phone','Address','Verified','Trusted','Url']) 
dfObj

,Name,Rating,Votes,Phone,Address,Verified,Trusted,Url
0,Omega RMC,4.5,15,+(91)-9152182118,"Flat No.6-150, Near Gandimaisamma Chowrasta, B...",True,True,https://www.justdial.com/hyderabad/Omega-RMC-B...
1,Premier Readymix Concrete,4.7,22,+(91)-9152874536,"Plot No 57, Telcom Nagar, Gachibowli, Hyderaba...",False,False,https://www.justdial.com/Hyderabad/Premier-Rea...
2,Rdc Concrete India Pvt Ltd,4.1,16,+(91)-9152886244,"Plot No 8/A, Survey No 334 to 337, Bollaram Ro...",False,False,https://www.justdial.com/Hyderabad/Rdc-Concret...
3,Metro Rmc,4.0,21,+(91)-9152194066,"39-p , 1ST Floor , B R R Residency ,D Pochamp...",True,True,https://www.justdial.com/Hyderabad/Metro-Rmc-N...
4,G T Associates,4.5,22,+(91)-9152861461,"4-55, Kukatpally, Hyderabad - 500072, Near BJP...",False,False,https://www.justdial.com/Hyderabad/G-T-Associa...
5,Bmr Ready Mix Concrete,4.9,8,+(91)-9152596296,"201,Chaitanya Chambers, Chaitanyapuri, Hyderab...",False,False,https://www.justdial.com/hyderabad/Bmr-Ready-M...
6,Mega Ready Mix Concrete,4.5,3,+(91)-9152217638,"Survey No 5-53, Kolluru Village, Tellapur, Hyd...",False,False,https://www.justdial.com/Hyderabad/Mega-Ready-...
7,Global Rmc,4.5,30,+(91)-9152235557,"Sy No.294, Rampalli Village, Cherlapalli, Rang...",False,False,https://www.justdial.com/Hyderabad/Global-Rmc-...
8,Jaidhar Ready Mix Concrete,4.2,63,+(91)-9152188052,"Plot No 37, A Colony, Karkhana, Hyderabad - 50...",False,False,https://www.justdial.com/Hyderabad/Jaidhar-Rea...
9,Sln Ready Mix Concrete,4.3,6,+(91)-9152883544,"House No 8/73/20, ECIL, Hyderabad - 500062, Ne...",False,False,https://www.justdial.com/Hyderabad/Sln-Ready-M...


In [34]:
startTime = time.time()
dfObj['Products'] = dfObj['Url'].apply(lambda x : getProducts(x))
print(time.time() - startTime)

443.23374485969543


In [35]:
dfObj

,Name,Rating,Votes,Phone,Address,Verified,Trusted,Url,Products
0,Omega RMC,4.5,15,+(91)-9152182118,"Flat No.6-150, Near Gandimaisamma Chowrasta, B...",True,True,https://www.justdial.com/hyderabad/Omega-RMC-B...,Ready Mix Concrete Dealers
1,Premier Readymix Concrete,4.7,22,+(91)-9152874536,"Plot No 57, Telcom Nagar, Gachibowli, Hyderaba...",False,False,https://www.justdial.com/Hyderabad/Premier-Rea...,"Ready Mix Concrete Dealers, Ready Mix Concrete..."
2,Rdc Concrete India Pvt Ltd,4.1,16,+(91)-9152886244,"Plot No 8/A, Survey No 334 to 337, Bollaram Ro...",False,False,https://www.justdial.com/Hyderabad/Rdc-Concret...,"Ready Mix Concrete Dealers, Ready Mix Concrete..."
3,Metro Rmc,4.0,21,+(91)-9152194066,"39-p , 1ST Floor , B R R Residency ,D Pochamp...",True,True,https://www.justdial.com/Hyderabad/Metro-Rmc-N...,"Ready Mix Concrete Dealers, Ready Mix Concrete..."
4,G T Associates,4.5,22,+(91)-9152861461,"4-55, Kukatpally, Hyderabad - 500072, Near BJP...",False,False,https://www.justdial.com/Hyderabad/G-T-Associa...,"Ready Mix Concrete Dealers, Cement Dealers-Ult..."
5,Bmr Ready Mix Concrete,4.9,8,+(91)-9152596296,"201,Chaitanya Chambers, Chaitanyapuri, Hyderab...",False,False,https://www.justdial.com/hyderabad/Bmr-Ready-M...,"Ready Mix Concrete Dealers, Ready Mix Concrete..."
6,Mega Ready Mix Concrete,4.5,3,+(91)-9152217638,"Survey No 5-53, Kolluru Village, Tellapur, Hyd...",False,False,https://www.justdial.com/Hyderabad/Mega-Ready-...,"Ready Mix Concrete Dealers, Concrete Mixer Dea..."
7,Global Rmc,4.5,30,+(91)-9152235557,"Sy No.294, Rampalli Village, Cherlapalli, Rang...",False,False,https://www.justdial.com/Hyderabad/Global-Rmc-...,"Ready Mix Concrete Dealers, Ready Mix Concrete..."
8,Jaidhar Ready Mix Concrete,4.2,63,+(91)-9152188052,"Plot No 37, A Colony, Karkhana, Hyderabad - 50...",False,False,https://www.justdial.com/Hyderabad/Jaidhar-Rea...,"Concrete Mixer-Birla Readymix, Ready Mix Concr..."
9,Sln Ready Mix Concrete,4.3,6,+(91)-9152883544,"House No 8/73/20, ECIL, Hyderabad - 500062, Ne...",False,False,https://www.justdial.com/Hyderabad/Sln-Ready-M...,"Ready Mix Concrete Dealers, Ready Mix Concrete..."
